<a href="https://colab.research.google.com/github/cyclingdata/kaggle_fraud_modelling/blob/master/Kaggle_fraud_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import pickle

In [0]:
gcs_data = "gs://data-fraud-detection/train_identity.csv"

In [0]:
# Authenticate.
from google.colab import auth
auth.authenticate_user()

In [0]:
!gsutil cp $gcs_data train.csv
dataset_identity=pd.read_csv('train.csv', low_memory=False)
!rm train.csv

Copying gs://data-fraud-detection/train_identity.csv...
\ [1 files][ 25.3 MiB/ 25.3 MiB]                                                
Operation completed over 1 objects/25.3 MiB.                                     


In [0]:
dataset_identity["device_info_cut"]=dataset_identity.DeviceInfo.str[:2]
dataset_identity['present_identity'] = 1

In [0]:
dataset_identity.columns

Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14',
       'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22',
       'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo', 'device_info_cut', 'present_identity'],
      dtype='object')

In [0]:
gcs_data = "gs://data-fraud-detection/train_transaction.csv"
!gsutil cp $gcs_data train.csv
dataset_transaction=pd.read_csv('train.csv', low_memory=False)

!rm train.csv

Copying gs://data-fraud-detection/train_transaction.csv...
/ [1 files][651.7 MiB/651.7 MiB]   70.7 MiB/s                                   
Operation completed over 1 objects/651.7 MiB.                                    


In [0]:
dataset_transaction = pd.merge(dataset_transaction,dataset_identity,on="TransactionID",how='left')

In [0]:
dataset_transaction.shape

(590540, 436)

In [0]:
dataset_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,device_info_cut,present_identity
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,SA,1.0


In [0]:
import datetime
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
dataset_transaction['Transaction_hour'] = np.floor(dataset_transaction['TransactionDT'] / 3600) % 24
dataset_transaction['TransactionDT'] = dataset_transaction['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
dataset_transaction['TransactionDT_Days'] = dataset_transaction['TransactionDT'].dt.day

In [0]:
import re
dataset_transaction['D9'] = dataset_transaction['TransactionDT'].dt.hour
dataset_transaction['dow'] = dataset_transaction['TransactionDT'].dt.dayofweek
dataset_transaction['id_30_alpha'] = dataset_transaction['id_30'].str.replace('[^a-zA-Z]', '')
dataset_transaction['id_31_alpha'] = dataset_transaction['id_31'].str.replace('[^a-zA-Z]', '')
dataset_transaction['group_os'] = dataset_transaction["id_30"].str.split(' ', expand = True)[[0]]
dataset_transaction['group_browser'] = dataset_transaction["id_31"].str.split(' ', expand = True)[[0]]

dataset_transaction['uid'] = dataset_transaction['card1'].astype(str)+'_'+dataset_transaction['card2'].astype(str)
dataset_transaction['uid2'] = dataset_transaction['uid'].astype(str)+'_'+dataset_transaction['card3'].astype(str)+'_'+dataset_transaction['card5'].astype(str)
dataset_transaction['uid3'] = dataset_transaction['uid2'].astype(str)+'_'+dataset_transaction['addr1'].astype(str)+'_'+dataset_transaction['addr2'].astype(str)


dataset_transaction['present_identity'].fillna(0,inplace=True)

In [0]:
dataset_transaction['device_name'] = dataset_transaction['DeviceInfo'].str.split('/', expand=True)[0]
dataset_transaction['device_version'] = dataset_transaction['DeviceInfo'].str.split('/', expand=True)[1] 

In [0]:
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC' 
dataset_transaction.loc[dataset_transaction['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'
dataset_transaction.loc[dataset_transaction.device_name.isin(dataset_transaction.device_name.value_counts()[dataset_transaction.device_name.value_counts() < 200].index), 'device_name'] = "Others"

In [0]:
dataset_transaction['Dollars'] = dataset_transaction['TransactionAmt'].apply(np.floor).astype('int')
dataset_transaction['Cents'] = (dataset_transaction['TransactionAmt'].round(2)*100- 100*dataset_transaction['Dollars']).astype('int')
dataset_transaction['Nb_decimals_cents'] = (dataset_transaction['TransactionAmt']-dataset_transaction['Dollars']).astype('str').str.len()
dataset_transaction['Dollars_last_digit'] = dataset_transaction['Dollars'].mod(10)  
dataset_transaction['Dollars_first_digit'] = dataset_transaction['Dollars'].astype('str').str[0] 
dataset_transaction[['Dollars','Cents','TransactionAmt','Nb_decimals_cents','Dollars_last_digit','Dollars_first_digit']].head(10)

,Dollars,Cents,TransactionAmt,Nb_decimals_cents,Dollars_last_digit,Dollars_first_digit
0,68,50,68.5,3,8,6
1,29,0,29.0,3,9,2
2,59,0,59.0,3,9,5
3,50,0,50.0,3,0,5
4,50,0,50.0,3,0,5
5,49,0,49.0,3,9,4
6,159,0,159.0,3,9,1
7,422,50,422.5,3,2,4
8,15,0,15.0,3,5,1
9,117,0,117.0,3,7,1


In [0]:
dataset_transaction.Nb_decimals_cents.value_counts()

3     326385
18    230868
19     23812
17      5027
20      2760
5        613
4        537
21       255
16       224
15        59
Name: Nb_decimals_cents, dtype: int64

In [0]:
dataset_transaction.loc[dataset_transaction.Nb_decimals_cents == 15,['Dollars','Cents','TransactionAmt','Nb_decimals_cents']].head()

,Dollars,Cents,TransactionAmt,Nb_decimals_cents
14551,1105,64,1105.64,15
24957,1838,89,1838.89,15
26704,1209,85,1209.86,15
44553,1803,61,1803.61,15
44998,1803,61,1803.61,15


In [0]:
dataset_transaction['screen_width'] = (dataset_transaction['id_33'].str.split('x', expand=True)[0]).astype(float)
dataset_transaction['screen_height'] = (dataset_transaction['id_33'].str.split('x', expand=True)[1]).astype(float)

In [0]:
dataset_transaction['id_31_alpha'].value_counts()

chrome                     49855
chromeforandroid           19701
mobilesafari               16797
mobilesafarigeneric        11474
iefordesktop                9030
safarigeneric               8195
firefox                     6901
edge                        6401
chromegeneric               4778
samsungbrowser              1818
chromegenericforandroid     1158
safari                       718
iefortablet                  647
chromeforios                 567
opera                        371
other                        312
androidwebview               285
samsungbrowsergeneric        210
SamsungSMGM                  150
GenericAndroid               138
firefoxgeneric               110
googlesearchapplication      110
androidbrowser               100
mobilesafariuiwebview         97
operageneric                  78
ie                            56
SamsungSMGH                   52
google                        36
MicrosoftWindows              25
silk                          19
samsung   

In [0]:
def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)

categorical_features = ['card1', 'card2', 'card3','card4','card5','card6','addr1','addr2','P_emaildomain','R_emaildomain','ProductCD','M1','M2','M3','M4','M5','M6','M7','M8','M9',
                        'DeviceType','DeviceInfo',"device_info_cut","dow",'id_30_alpha','id_31_alpha','group_os','group_browser','uid','uid2','uid3','TransactionDT_Days','device_name',
                        'Dollars_first_digit','Dollars_last_digit','Cents','Dollars'
                        ]
numerical_features = ['TransactionAmt','dist1','dist2','C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','D1','D2','D3','D4','D5','D6','D7','D8','D9',
                      'id_01','id_02','id_03', 'id_04','id_05', 'id_06', 'id_07','id_08','id_09','id_10','id_11','screen_width','screen_height','present_identity','Nb_decimals_cents', 'Transaction_hour'
                     ]
categorical_count_only = ['uid','uid2', 'uid3']

In [0]:
for val in range(12,39):
  categorical_features.append("id_"+str(val))
for val in range(1,340):
  numerical_features.append("V"+str(val))

In [0]:
y = dataset_transaction.isFraud
X= dataset_transaction.drop("isFraud",axis=1)
X['index_row'] = X.index

In [0]:
from sklearn.model_selection import train_test_split

#split = X.TransactionDT.quantile(0.75)
#istrain = X.TransactionDT < split
#X_train = X.loc[istrain]
#X_test = X.loc[~istrain]

#y_train_values = y[istrain]
#y_test_values = y[~istrain]
X_train, X_test, y_train_values, y_test_values = train_test_split(X,y, test_size=0.2, random_state=42)

In [0]:
to_impute = {}
import math
for feature in numerical_features:
  value_to_impute = X_train[feature].min() - 1
  if math.isnan(value_to_impute):
    numerical_features.remove(feature)
  else:
    X_train[feature].fillna(value_to_impute,inplace=True)
    X_test[feature].fillna(value_to_impute,inplace=True)
    to_impute[feature] = value_to_impute
print(to_impute)

#numerical_features = to_impute.keys().to_list()

for feature in categorical_features:
  X_train[feature] = X_train[feature].astype("str")
  X_test[feature] = X_test[feature].astype("str")
  X_train[feature]=X_train[feature].str.replace(' ', '_')
  X_test[feature]=X_test[feature].str.replace(' ','_')
  X_train[feature]=X_train[feature].str.replace('.0', '',regex=False)
  X_test[feature]=X_test[feature].str.replace('.0','',regex=False)
  X_train[feature]=X_train[feature].str.replace('.', '_',regex=False)
  X_test[feature]=X_test[feature].str.replace('.', '_',regex=False)
  X_train[feature]=X_train[feature].str.replace(':','_',regex=False)
  X_test[feature]=X_test[feature].str.replace(':','_',regex=False)
  X_test[feature]=X_test[feature].str.replace('/','_',regex=False)
  X_train[feature]=X_train[feature].str.replace('/','_',regex=False)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


{'TransactionAmt': -0.728, 'dist1': -1.0, 'dist2': -1.0, 'C1': -1.0, 'C2': -1.0, 'C3': -1.0, 'C4': -1.0, 'C5': -1.0, 'C6': -1.0, 'C7': -1.0, 'C8': -1.0, 'C9': -1.0, 'C10': -1.0, 'C11': -1.0, 'C12': -1.0, 'C13': -1.0, 'C14': -1.0, 'D1': -1.0, 'D2': -1.0, 'D3': -1.0, 'D4': -123.0, 'D5': -1.0, 'D6': -84.0, 'D7': -1.0, 'D8': -1.0, 'D9': -1, 'id_01': -101.0, 'id_02': 0.0, 'id_03': -14.0, 'id_04': -29.0, 'id_05': -73.0, 'id_06': -101.0, 'id_07': -47.0, 'id_08': -101.0, 'id_09': -37.0, 'id_10': -101.0, 'id_11': 89.0, 'screen_width': -1.0, 'screen_height': -1.0, 'present_identity': -1.0, 'Nb_decimals_cents': 2, 'Transaction_hour': -1.0, 'V1': -1.0, 'V2': -1.0, 'V3': -1.0, 'V4': -1.0, 'V5': -1.0, 'V6': -1.0, 'V7': -1.0, 'V8': -1.0, 'V9': -1.0, 'V10': -1.0, 'V11': -1.0, 'V12': -1.0, 'V13': -1.0, 'V14': -1.0, 'V15': -1.0, 'V16': -1.0, 'V17': -1.0, 'V18': -1.0, 'V19': -1.0, 'V20': -1.0, 'V21': -1.0, 'V22': -1.0, 'V23': -1.0, 'V24': -1.0, 'V25': -1.0, 'V26': -1.0, 'V27': -1.0, 'V28': -1.0, 'V29': -

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

In [0]:
min_cat = 0.0005
valid_values = {}
dict_freq_cat = {}
old_numerical_features=numerical_features.copy()
for cat in categorical_features:

    X_train[cat].fillna('missing',inplace=True)
    X_test[cat].fillna('missing',inplace=True)
    counts=X_train[cat].value_counts()/X_train.shape[0]
    #rare_values = counts[counts <= min_cat].index.tolist()
    #X_train[cat].replace(rare_values, "RARE_VALUE",inplace=True)
    counts = counts.rename_axis(cat).reset_index(name='count_'+cat)
    dict_freq_cat[cat] = counts
    X_train[cat] = X_train[cat].astype('str')
    X_train = pd.merge(X_train,counts,on=cat,how='left')
    X_train['count_'+cat].fillna(0,inplace=True)
    X_test[cat] = X_test[cat].astype('str')
    X_test = pd.merge(X_test,counts,on=cat,how='left')
    X_test['count_'+cat].fillna(0,inplace=True)
    counts=X_train[cat].value_counts()
    numerical_features.append('count_'+cat)
    valid_values[cat] = counts.index.tolist()
    print(cat + " done")
    print(valid_values[cat])
    
categorical_features = valid_values

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


card1 done
['7919', '9500', '17188', '15885', '15066', '12695', '12544', '6019', '2803', '7585', '10616', '12839', '3154', '2616', '18132', '9633', '15497', '16132', '2884', '16075', '11207', '10112', '7508', '10057', '12501', '7826', '16659', '12577', '5812', '7664', '7861', '4461', '6530', '8320', '5033', '15063', '7207', '10486', '10023', '7815', '9112', '16560', '9026', '13832', '16136', '7676', '4806', '12932', '11839', '1444', '4436', '2772', '17399', '2455', '1675', '14290', '11157', '3821', '17055', '1893', '17131', '18018', '1974', '1724', '9803', '13249', '2392', '8528', '15775', '17400', '15651', '11201', '14858', '3682', '18249', '2256', '12686', '15484', '1342', '12469', '9992', '1939', '6550', '9175', '9300', '4663', '16255', '8695', '6174', '8755', '18268', '8406', '4272', '6170', '10989', '11106', '9002', '15986', '16062', '9480', '13780', '13481', '6741', '5938', '13108', '10447', '10960', '13307', '11218', '4504', '5700', '11919', '10568', '1976', '10856', '2581', '40

In [0]:
#X_train_new.to_csv('X_train_new_python.csv')
#X_test_new.to_csv('X_test_new_python.csv')
#!gsutil cp X_train_new_python.csv gs://data-fraud-detection/X_train_new_python.csv
#!gsutil cp X_test_new_python.csv gs://data-fraud-detection/X_test_new_python.csv
#pickle.dump(X_train_new, open( "X_train_new.pickle", "wb" ) )
#pickle.dump(X_test_new, open( "X_test_new.pickle", "wb" ) )
#!gsutil cp X_train_new.pickle gs://data-fraud-detection/X_train_new.pickle
#!gsutil cp X_test_new.pickle gs://data-fraud-detection/X_test_new.pickle
pickle.dump(y_train_values, open( "y_train_values.pickle", "wb" ) )
pickle.dump(y_test_values, open( "y_test_values.pickle", "wb" ) )
!gsutil cp y_train_values.pickle gs://data-fraud-detection/y_train_values.pickle
!gsutil cp y_test_values.pickle gs://data-fraud-detection/y_test_values.pickle

Copying file://y_train_values.pickle [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/10.8 MiB.                                     
Copying file://y_test_values.pickle [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/2.7 MiB.                                      


In [0]:
pickle.dump(to_impute, open( "to_impute.pickle", "wb" ) )
pickle.dump(dict_freq_cat, open( "dict_freq_cat.pickle", "wb" ) )
!gsutil cp to_impute.pickle gs://data-fraud-detection/to_impute.pickle
!gsutil cp dict_freq_cat.pickle gs://data-fraud-detection/dict_freq_ca.pickle

Copying file://to_impute.pickle [Content-Type=application/octet-stream]...
/ [1 files][ 16.1 KiB/ 16.1 KiB]                                                
Operation completed over 1 objects/16.1 KiB.                                     
Copying file://dict_freq_cat.pickle [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/2.9 MiB.                                      


In [0]:
pickle.dump(categorical_features, open( "categorical_features.pickle", "wb" ) )
pickle.dump(numerical_features, open( "numerical_features.pickle", "wb" ) )
pickle.dump(categorical_count_only, open( "categorical_count_only.pickle", "wb" ) )
!gsutil cp numerical_features.pickle gs://data-fraud-detection/numerical_features.pickle
!gsutil cp categorical_features.pickle gs://data-fraud-detection/categorical_features.pickle
!gsutil cp categorical_count_only.pickle gs://data-fraud-detection/categorical_count_only.pickle

Copying file://numerical_features.pickle [Content-Type=application/octet-stream]...
/ [1 files][  5.8 KiB/  5.8 KiB]                                                
Operation completed over 1 objects/5.8 KiB.                                      
Copying file://categorical_features.pickle [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/2.2 MiB.                                      
Copying file://categorical_count_only.pickle [Content-Type=application/octet-stream]...
/ [1 files][   40.0 B/   40.0 B]                                                
Operation completed over 1 objects/40.0 B.                                       


In [0]:
X_train.to_parquet('X_train_cat.parquet')
!gsutil cp X_train_cat.parquet gs://data-fraud-detection/X_train_cat.parquet
X_test.to_parquet('X_test_cat.parquet')
!gsutil cp X_test_cat.parquet gs://data-fraud-detection/X_test_cat.parquet

Copying file://X_train_cat.parquet [Content-Type=application/octet-stream]...
/
Operation completed over 1 objects/103.2 MiB.                                    
Copying file://X_test_cat.parquet [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/26.9 MiB.                                     


In [0]:
total = numerical_features.copy()
total.append('index_row')
print(total[-1])
X_train_new = X_train[total]
X_test_new = X_test[total]
print('debut boucle')
for feature in categorical_features.keys():
  if feature not in categorical_count_only:
    for category in categorical_features[feature]:
        new_feature = feature + "_" + category
        print(new_feature)
        X_train_new[new_feature]=np.where(X_train[feature]==category,1,0)
        X_test_new[new_feature]=np.where(X_test[feature]==category,1,0)

index_row
debut boucle
card1_RARE_VALUE


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


card1_7919
card1_9500
card1_17188
card1_15885
card1_15066
card1_12695
card1_12544
card1_6019
card1_2803
card1_7585
card1_10616
card1_12839
card1_3154
card1_2616
card1_18132
card1_9633
card1_15497
card1_16132
card1_2884
card1_16075
card1_11207
card1_10112
card1_7508
card1_10057
card1_12501
card1_7826
card1_16659
card1_12577
card1_5812
card1_7664
card1_7861
card1_4461
card1_6530
card1_8320
card1_5033
card1_15063
card1_7207
card1_10486
card1_10023
card1_7815
card1_9112
card1_16560
card1_9026
card1_13832
card1_16136
card1_7676
card1_4806
card1_12932
card1_11839
card1_1444
card1_4436
card1_2772
card1_17399
card1_2455
card1_1675
card1_14290
card1_11157
card1_17055
card1_3821
card1_1893
card1_17131
card1_18018
card1_1974
card1_1724
card1_9803
card1_13249
card1_2392
card1_8528
card1_15775
card1_17400
card1_15651
card1_11201
card1_14858
card1_3682
card1_18249
card1_2256
card1_12686
card1_15484
card1_1342
card1_12469
card1_9992
card1_1939
card1_6550
card1_9175
card1_9300
card1_4663
card1_16255
c

In [0]:
X_train_new['index_row'][:5]

0      5307
1    191582
2    260168
3     18516
4     47538
Name: index_row, dtype: int64

In [0]:
y_train_values.index[:5]

Int64Index([5307, 191582, 260168, 18516, 47538], dtype='int64')

In [0]:
X_test_new.columns

Index(['TransactionAmt', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6',
       'C7',
       ...
       'id_35_F', 'id_36_nan', 'id_36_F', 'id_36_T', 'id_37_nan', 'id_37_T',
       'id_37_F', 'id_38_nan', 'id_38_F', 'id_38_T'],
      dtype='object', length=1573)

In [0]:
X_train_new.shape

(472432, 1573)

In [0]:
df_y_train = pd.DataFrame({'index_row': y_train_values.index,'label': y_train_values.tolist()})
df_y_test = pd.DataFrame({'index_row': y_test_values.index,'label': y_test_values.tolist()})
X_train_new = pd.merge(X_train_new,df_y_train,how='left',on='index_row')
X_test_new = pd.merge(X_test_new,df_y_train,how='left',on='index_row')
#pickle.dump(X_train_new, open( "X_train_new_pyspark.pickle", "wb" ) )
#pickle.dump(X_test_new, open( "X_test_new_pyspark.pickle", "wb" ) )
#!gsutil cp X_train_new_pyspark.pickle gs://data-fraud-detection/X_train_new_pyspark.pickle
#!gsutil cp X_test_new_pyspark.pickle gs://data-fraud-detection/X_test_new_pyspark.pickle

In [0]:
X_train_new['index_row'][:5]

0      5307
1    191582
2    260168
3     18516
4     47538
Name: index_row, dtype: int64

In [0]:
len(y_train_values)

472432

In [0]:
from collections import Counter
Counter(y_train_values)

Counter({0: 456011, 1: 16421})

In [0]:
X_train_new['label'].head(5)

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [0]:
y_train_values[:4]

5307      0
191582    0
260168    0
18516     0
Name: isFraud, dtype: int64

In [0]:
X_train_new.label.value_counts()

0    456011
1     16421
Name: label, dtype: int64

In [0]:
X_train_new.shape

(472432, 1574)

In [0]:
y_train_values.isnull().sum()

0

In [0]:
X_train_new.to_parquet('X_train_new.parquet')
!gsutil cp X_train_new.parquet gs://data-fraud-detection/X_train_new.parquet
X_test_new.to_parquet('X_test_new.parquet')
!gsutil cp X_test_new.parquet gs://data-fraud-detection/X_test_new.parquet

Copying file://X_train_new.parquet [Content-Type=application/octet-stream]...
\
Operation completed over 1 objects/93.3 MiB.                                     
Copying file://X_test_new.parquet [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/24.5 MiB.                                     


In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_features=30,n_estimators=100,min_samples_split=100,min_samples_leaf =50,n_jobs=-1)
#from sklearn.tree import DecisionTreeClassifier
#rf= DecisionTreeClassifier(min_samples_split=500,min_samples_leaf=200)

#from xgboost import XGBRegressor
#xgb=XGBRegressor(n_estimator=100,min_child_weight=100,colsample_bynode=0.15)

In [0]:
%%time
rf.fit(X_train_new,y_train_values)

In [0]:
pred = rf.predict_proba(X_test_new)[:,1].tolist()

In [0]:
from sklearn.metrics import roc_curve, auc

In [0]:
fpr,tpr,threshold=roc_curve(y_test_values,pred)

In [0]:
plt.plot(fpr,tpr)

In [0]:
auc(fpr,tpr)

In [0]:
gcs_data = "gs://data-fraud-detection/test_transaction.csv"
!gsutil cp $gcs_data test.csv
test_transaction=pd.read_csv('test.csv', low_memory=False)
!rm test.csv

In [0]:
gcs_data = "gs://data-fraud-detection/test_identity.csv"
!gsutil cp $gcs_data test.csv
test_identity=pd.read_csv('test.csv', low_memory=False)
!rm test.csv

In [0]:
test_identity["device_info_cut"]=test_identity.DeviceInfo.str[:2]

In [0]:
test_transaction = pd.merge(test_transaction,test_identity,on="TransactionID",how='left')

In [0]:
test_transaction['TransactionDT'] = test_transaction['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))


test_transaction['D9'] = test_transaction['TransactionDT'].dt.hour 
test_transaction['dow'] = test_transaction['TransactionDT'].dt.dayofweek

In [0]:
for feature in to_impute.keys():
  test_transaction[feature].fillna(to_impute[feature],inplace=True)
for feature in categorical_features:
  test_transaction[feature] = test_transaction[feature].astype("str")

In [0]:

for feature in categorical_features.keys():
    test_transaction = pd.merge(test_transaction,dict_freq_cat[feature],on=feature,how="left")
    test_transaction['count_'+feature].fillna(0,inplace=True)
    
    
test_new = test_transaction[numerical_features]
for feature in categorical_features.keys():
    for category in categorical_features[feature]:
        new_feature = feature + "_" + category
        test_new[new_feature]=np.where(test_transaction[feature]==category,1,0)

In [0]:
dict_freq_cat[feature]

In [0]:
test_new.isnull().sum()

In [0]:
pred_test= rf.predict_proba(test_new)[:,1].tolist()

In [0]:
submission=pd.DataFrame({"TransactionID": test_transaction.TransactionID,"isFraud":pred_test})

In [0]:
submission.head()

In [0]:
submission.to_csv("submission.csv",index=False)

In [0]:
!gsutil cp submission.csv gs://data-fraud-detection/submission.csv